In [ ]:
%load_ext autoreload 
%autoreload 2
import numpy as np
import random
from colossus.cosmology import cosmology
cosmology.setCosmology('planck18')
from colossus.halo import concentration, mass_defs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from pylab import rcParams
from matplotlib import rc;rc('text', usetex=True);rc('font', weight='bold');matplotlib.rcParams['text.latex.preamble'] = r'\boldmath'
rcParams['font.family'] = 'serif'
rc('text.latex',preamble=r'\usepackage{apjfonts}')
sz_ft = 20
sz_lb = 14
color_arr = ['indigo', 'royalblue', 'lightseagreen', 'darkgreen', 'goldenrod', 'darkred']
from tqdm import tqdm
import lensing_estimator
from cosmo import CosmoCalc
import lensing
import foregrounds as fg
import experiments as exp
import sims
import stats
import tools

In [ ]:
nber_clus = 3000
nber_rand = 30000
nber_cov = 500
map_params = [240, 0.25, 240, 0.25]
l, cl = CosmoCalc().cmb_power_spectrum()
l, bl = exp.beam_power_spectrum(beam_fwhm = 1.4)
l, nl = exp.white_noise_power_spectrum(noiseval_white = 2.0)
l, nl_deconvolved =  exp.white_noise_power_spectrum(noiseval_white = 2.0, beam_fwhm = 1.4)
cl_noise =  nl_deconvolved
nber_clus_fit = 30000
nber_rand_fit = 30000
mass_int_estimator_validation = np.arange(0, 12, 0.1)
c200c = 3
z = 1


covariance_matrix_estimator_validation, correlation_matrix_estimator_validation = lensing_estimator.covariance_and_correlation_matrix(nber_cov, nber_clus, map_params, l, cl, bl = bl, nl = nl, cl_noise = cl_noise)
with open('covariance_matrix_estimator_validation.npy', 'wb') as file:
    np.save(file, covariance_matrix_estimator_validation)   
        

model_profiles_estimator_validation = lensing_estimator.model_profiles(nber_clus_fit, nber_rand_fit, map_params, l, cl, mass_int_estimator_validation, c200c, z, bl = bl, cl_noise = cl_noise)
with open('model_profiles_estimator_validation.npy', 'wb') as file:
    np.save(file, model_profiles_estimator_validation)    

            
            
lk_arr_2e14 = []      
lk_arr_6e14 = []  
lk_arr_10e14 = []  
for i in tqdm(range(25)):    
    maps_2e14, maps_6e14, maps_10e14 = sims.cmb_test_data(nber_clus, validation_analyis = True)
    maps_rand = []
    for j in range(nber_rand):    
        map_rand = sims.cmb_mock_data(map_params, l, cl,  bl = bl, nl = nl)
        maps_rand.append(map_rand)
    bins_2e14, lensing_dipole_profile_2e14, stacks_2e14 = lensing_estimator.lensing_dipole_profile(map_params, maps_2e14, maps_rand, l = l, cl = cl, cl_noise = cl_noise)
    bins_6e14, lensing_dipole_profile_6e14, stacks_6e14 = lensing_estimator.lensing_dipole_profile(map_params, maps_6e14, maps_rand, l = l, cl = cl, cl_noise = cl_noise)
    bins_10e14, lensing_dipole_profile_10e14, stacks_10e14 = lensing_estimator.lensing_dipole_profile(map_params, maps_10e14, maps_rand, l = l, cl = cl, cl_noise = cl_noise)
    data_2e14 = bins_2e14, lensing_dipole_profile_2e14, covariance_matrix_estimator_validation
    data_6e14 = bins_6e14, lensing_dipole_profile_6e14, covariance_matrix_estimator_validation
    data_10e14 = bins_10e14, lensing_dipole_profile_10e14, covariance_matrix_estimator_validation
   

    likelihood_2e14, _, _ = stats.run_ml(data_2e14, model_profiles_estimator_validation, mass_int_estimator_validation)
    mass_int_lk_2e14, lk_2e14 = likelihood_2e14
    likelihood_6e14, _, _ = stats.run_ml(data_6e14, model_profiles_estimator_validation, mass_int_estimator_validation)
    mass_int_lk_6e14, lk_6e14 = likelihood_6e14
    likelihood_10e14, _, _ = stats.run_ml(data_10e14, model_profiles_estimator_validation, mass_int_estimator_validation)
    mass_int_lk_10e14, lk_10e14 = likelihood_10e14
    lk_arr_2e14.append(lk_2e14)
    lk_arr_6e14.append(lk_6e14)
    lk_arr_10e14.append(lk_10e14)

    del maps_2e14
    del maps_6e14
    del maps_10e14
    del maps_rand
    
    
with open('lk_arr_estimator_validation_2e14.npy', 'wb') as file:
    np.save(file, lk_arr_2e14) 
    
with open('lk_arr_estimator_validation_6e14.npy', 'wb') as file:
    np.save(file, lk_arr_6e14) 
    
with open('lk_arr_estimator_validation_10e14.npy', 'wb') as file:
    np.save(file, lk_arr_10e14) 